In [ ]:
#!pip3 install cassandra-driver

In [ ]:
# Importing packages

from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider
import csv
import uuid

In [ ]:
ASTRA_DB_ID = '19cf8906-f987-4b74-a9a9-1ca42c56e000'
ASTRA_DB_REGION = 'us-east1'
ASTRA_DB_APPLICATION_TOKEN = 'AstraCS:oooCMShTTrtnyocJlEpAnSEr:3233df889ee8dedfcc3c4a252d3e676ff09821828f40d0c247df5628f7e7e666'
ASTRA_DB_KEYSPACE = 'insurance_keyspace'
ASTRA_DB_COLLECTION= 'insurance_collection'
ASTRA_CLIENT_ID = 'oooCMShTTrtnyocJlEpAnSEr'
ASTRA_CLIENT_SECRET = '87ZyISoBoXZSkgtZfHIq1P-C9y+PzZpwHMQLfxXJ6n0SbyS9tJRx2+ju4h.1QCu,lNYzPZYhHm6K6M.c-oH7UMt27Z.yuvxUPU4czu48qlzQZvt5XZ4ePwpy6SSHBPd6'

In [ ]:
# Connecting to Astra Database

cloud_config = {'secure_connect_bundle': r"secure-connect-insurance.zip"}

cloud_config

In [ ]:
# Creating auth_provider

auth_provider = PlainTextAuthProvider(ASTRA_CLIENT_ID, ASTRA_CLIENT_SECRET)

auth_provider

In [ ]:
Cluster = Cluster(cloud= cloud_config, auth_provider= auth_provider)

Cluster

In [ ]:
session = Cluster.connect()

In [ ]:
drop_table_cql = f"DROP TABLE IF EXISTS {ASTRA_DB_KEYSPACE}.{ASTRA_DB_COLLECTION}"

In [ ]:
create_table_cql = f"CREATE TABLE IF NOT EXISTS {ASTRA_DB_KEYSPACE}.{ASTRA_DB_COLLECTION} (\
                    customer_id UUID PRIMARY KEY, \
                    age INT, \
                    sex VARCHAR, \
                    bmi FLOAT, \
                    children INT, \
                    smoker VARCHAR, \
                    region VARCHAR, \
                    expenses FLOAT);" 

In [ ]:
session.execute(drop_table_cql)
session.execute(create_table_cql)

In [ ]:
csv_file = "insurance.csv"

In [ ]:
# Preparing the INSERT INTO quary with placeholders

insert_query = f"INSERT INTO {ASTRA_DB_KEYSPACE}.{ASTRA_DB_COLLECTION} " \
                "(customer_id, age, sex, bmi, children, smoker, region, expenses)" \
                "VALUES(?, ?, ?, ?, ?, ?, ?, ?);"

In [ ]:

with open(csv_file, 'r') as file:
    csv_reader = csv.DictReader(file)
    for row in csv_reader:
        customer_id = uuid.uuid4()
        age = int(row["age"])
        sex = row['sex']
        bmi = float(row['bmi'])
        children = int(row['children'])
        smoker = row["smoker"]
        region = row['region']
        expenses = float(row['expenses'])

        # Execute the INSERT query with parameterized values
        prepared = session.prepare(insert_query)
        session.execute(prepared, (customer_id, age, sex, bmi, children, smoker, region, expenses))
    print("Success!")

In [ ]:
rows = session.execute(f"SELECT * FROM {ASTRA_DB_KEYSPACE}.{ASTRA_DB_COLLECTION}")

for row in rows[:20]:
    # customer_id, age, bmi, children, expenses, region, sex, smoker
    print(row[0], row[1], round(row[2],2), row[3], round(row[4],2), row[5], row[6], row[7])